In [1]:
# Import required libraries
import os
import json
import pandas as pd
import openreview
import getpass
from tqdm import tqdm
from openreview import OpenReviewException

# Task
Download ICLR 2024 and ICLR 2025 submission and review data from OpenReview, extract specific review scores (soundness, presentation, contribution, rating, confidence), and consolidate the extracted information into a pandas DataFrame, then display its head and info.

## Prepare Data Directories

### Subtask:
Ensure the necessary directories for ICLR 2024 and ICLR 2025 data exist in the mounted Google Drive. If not, create them.


**Reasoning**:
To ensure the necessary data directories exist, I will use the `os` module to define paths and create them in Google Drive if they don't already exist.



In [2]:
# Define the base path for your data directory (local)
base_path = os.path.join(os.getcwd(), "data")

# Define the paths for the ICLR 2024 and ICLR 2025 data directories
iclr_2024_path = os.path.join(base_path, "ICLR_2024")
iclr_2025_path = os.path.join(base_path, "ICLR_2025")

# Create the directories if they don't already exist
os.makedirs(iclr_2024_path, exist_ok=True)
os.makedirs(iclr_2025_path, exist_ok=True)

print(f"Ensured directory: {iclr_2024_path}")
print(f"Ensured directory: {iclr_2025_path}")

Ensured directory: /Users/ktgiahieu/Documents/peer-review-pareto-front/data/ICLR_2024
Ensured directory: /Users/ktgiahieu/Documents/peer-review-pareto-front/data/ICLR_2025


## Authenticate with OpenReview

### Subtask:
Log in to the OpenReview API to allow for downloading submissions and reviews.


**Reasoning**:
To authenticate with OpenReview, I need to import the `openreview` library, securely get the username and password, and then create an `openreview.Client` instance.



In [3]:
%load_ext dotenv
%dotenv

In [4]:
import os

# Get OpenReview credentials securely
print("Enter your OpenReview credentials:")
username = os.environ['OPENREVIEW_USERNAME']
password = os.environ['OPENREVIEW_PASSWORD']

# Instantiate the OpenReview client
client = openreview.api.OpenReviewClient(
    baseurl='https://api2.openreview.net', username=username, password=password)

print("OpenReview client initialized and authenticated.")

Enter your OpenReview credentials:
OpenReview client initialized and authenticated.


## Download ICLR 2024 and 2025 Data

### Subtask:
Download all submissions and their associated reviews for both ICLR 2024 and ICLR 2025 from OpenReview, storing them in the prepared directories.


**Reasoning**:
To download the ICLR 2024 and 2025 data, I need to use the authenticated OpenReview client to fetch submissions and reviews, then save them into the pre-defined directories as JSON files, incorporating progress bars and error handling as specified.



In [5]:
# Download ICLR 2024 and 2025 data and extract review scores into DataFrame

# Define the conferences to process
conferences = [
    {
        "name": "ICLR2024",
        "id": "ICLR.cc/2024/Conference",
    },
    {
        "name": "ICLR2025",
        "id": "ICLR.cc/2025/Conference",
    }
]

def extract_score(value):
    """Parses OpenReview scores which can be integers (8) or strings ('8: Accept')."""
    if isinstance(value, (int, float)):
        return int(value)
    if isinstance(value, str):
        # Handle cases like "8: Accept" or just "8"
        try:
            return int(value.split(':')[0].strip())
        except ValueError:
            try:
                return int(value.split()[0].strip())
            except ValueError:
                return None
    return None

# List to store all review data (one record per review)
all_reviews_data = []

for conference in conferences:
    conference_id = conference["id"]
    conference_name = conference["name"]
    print(f"\nProcessing conference: {conference_name} ({conference_id})")

    try:
        # Construct the submission invitation
        submission_invitation = f"{conference_id}/-/Submission"
        print(f"Fetching submissions for {conference_id}...")

        # Retrieve all submissions with replies (reviews) included - this is fast
        submissions = list(tqdm(
            client.get_all_notes(invitation=submission_invitation, details='replies'),
            desc=f"Fetching submissions for {conference_name}"
        ))
        print(f"Found {len(submissions)} submissions for {conference_name}.")

        # Process each submission
        for sub_dict in tqdm(submissions, desc=f"Extracting {conference_name} review scores"):
            replies = sub_dict.details.get('replies', [])
            
            
            # Extract submission status (accept/reject)
            status = None
            venue_id = sub_dict.content.get('venueid', {})
            if isinstance(venue_id, dict):
                venue_id = venue_id.get('value', '')
            venue_id_str = str(venue_id).lower()
            if 'withdrawn' in venue_id_str:
                status = 'Withdrawn'
            elif 'desk_reject' in venue_id_str or 'desk reject' in venue_id_str:
                status = 'Desk Reject'
            elif 'reject' in venue_id_str or venue_id_str == 'rejected':
                status = 'reject'
            elif conference_id.lower() in venue_id_str or 'accept' in venue_id_str:
                status = 'accept'
            
                
            # Access replies from details (reviews are already here, no API call needed!)
            replies = sub_dict.details.get('replies', [])
            
            # Process each reply (review or decision)
            for reply in replies:
                # Check if this is a review (not a decision or other note type)
                reply_invitations = reply.get('invitations', '')
                is_review = any(['Review' in invitation or 'Official_Review' in invitation for invitation in reply_invitations])

                # Check if this is a decision note
                is_decision = any(['Decision' in invitation for invitation in reply_invitations])
                
                # Extract status from decision notes if not already found
                if is_decision:
                    content = reply.get('content', {})
                    if 'decision' in content:
                        decision_val = content['decision']
                        if isinstance(decision_val, dict):
                            decision_val = decision_val.get('value', '')
                        decision_str = str(decision_val).lower()
                        if 'accept' in decision_str:
                            status = 'accept'
                        elif 'reject' in decision_str:
                            status = 'reject'
                # Extract review scores
                submission_id = sub_dict.id
                if is_review:
                    review_data = {
                        'conference': conference_name,
                        'submission_id': submission_id,
                        'review_id': reply.get('id', None),
                    }
                    
                    # Extract scores from review content
                    content = reply.get('content', {})
                    
                    # Extract each score type
                    for score_type in ['soundness', 'presentation', 'contribution', 'rating', 'confidence']:
                        if score_type in content:
                            score_val = content[score_type]
                            if isinstance(score_val, dict):
                                score_val = score_val.get('value', score_val)
                            review_data[score_type] = extract_score(score_val)
                        else:
                            review_data[score_type] = None
                    
                    # Add status (same for all reviews of the same submission)
                    review_data['status'] = status if status else 'Pending/Unknown'
                    
                    all_reviews_data.append(review_data)

        print(f"Successfully processed {conference_name}.")

    except OpenReviewException as e:
        print(f"Error fetching data for {conference_id}: {e}")
        print("Consider checking your OpenReview credentials, API limits, or the conference invitation IDs.")
    except Exception as e:
        print(f"An unexpected error occurred while processing {conference_id}: {e}")
        import traceback
        traceback.print_exc()

print(f"\nTotal reviews extracted: {len(all_reviews_data)}")

# Create DataFrame
df = pd.DataFrame(all_reviews_data)

# Display head and info
print("\n" + "="*80)
print("DataFrame Head:")
print("="*80)
print(df.head())

print("\n" + "="*80)
print("DataFrame Info:")
print("="*80)
df.info()

print("\n" + "="*80)
print("DataFrame Shape:")
print("="*80)
print(f"Rows: {len(df)}, Columns: {len(df.columns)}")

# Store the DataFrame for later use
print("\nDataFrame stored in variable 'df'")


Processing conference: ICLR2024 (ICLR.cc/2024/Conference)
Fetching submissions for ICLR.cc/2024/Conference...


Fetching submissions for ICLR2024: 100%|██████████| 7404/7404 [00:00<00:00, 6119138.29it/s]


Found 7404 submissions for ICLR2024.


Extracting ICLR2024 review scores: 100%|██████████| 7404/7404 [00:00<00:00, 34562.94it/s]


Successfully processed ICLR2024.

Processing conference: ICLR2025 (ICLR.cc/2025/Conference)
Fetching submissions for ICLR.cc/2025/Conference...


Fetching submissions for ICLR2025: 100%|██████████| 11672/11672 [00:00<00:00, 4107039.96it/s]


Found 11672 submissions for ICLR2025.


Extracting ICLR2025 review scores: 100%|██████████| 11672/11672 [00:00<00:00, 61300.56it/s]


Successfully processed ICLR2025.

Total reviews extracted: 89284

DataFrame Head:
  conference submission_id   review_id  soundness  presentation  contribution  \
0   ICLR2024    zzv4Bf50RW  Upm4GL7iRr        3.0           3.0           2.0   
1   ICLR2024    zzv4Bf50RW  PUauntqQsB        3.0           3.0           3.0   
2   ICLR2024    zzv4Bf50RW  NLfuL11uwO        2.0           2.0           2.0   
3   ICLR2024    zzv4Bf50RW  wKVoAVtxek        3.0           3.0           2.0   
4   ICLR2024    zzv4Bf50RW  aU4SaZOeh4        3.0           2.0           3.0   

   rating  confidence     status  
0     5.0         4.0  Withdrawn  
1     5.0         4.0  Withdrawn  
2     3.0         4.0  Withdrawn  
3     5.0         4.0  Withdrawn  
4     6.0         2.0  Withdrawn  

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89284 entries, 0 to 89283
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   con

In [ ]:
df.to_csv('data/ICLR_reviews_data.csv', index=False)

In [7]:
df.head()

,conference,submission_id,review_id,soundness,presentation,contribution,rating,confidence,status
0,ICLR2024,zzv4Bf50RW,Upm4GL7iRr,3.0,3.0,2.0,5.0,4.0,Withdrawn
1,ICLR2024,zzv4Bf50RW,PUauntqQsB,3.0,3.0,3.0,5.0,4.0,Withdrawn
2,ICLR2024,zzv4Bf50RW,NLfuL11uwO,2.0,2.0,2.0,3.0,4.0,Withdrawn
3,ICLR2024,zzv4Bf50RW,wKVoAVtxek,3.0,3.0,2.0,5.0,4.0,Withdrawn
4,ICLR2024,zzv4Bf50RW,aU4SaZOeh4,3.0,2.0,3.0,6.0,2.0,Withdrawn
